# Example of GA using DEAP

Here i will try to understand how deap works. The task is a regressor, done with a tree that encodes a function.

In [1]:
from deap import algorithms, base, creator, tools, gp
import operator, random
import numpy as np

Individual and fitness creatio, the weight is -1 because is a minimization problem. Remeber the tuple!

In [2]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

Creation of a primitive set from which the tree will be generated.

In [3]:
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, arity=2 )
pset.addPrimitive(operator.sub, arity=2)
pset.addPrimitive(operator.mul, arity=2)
pset.addEphemeralConstant("const", lambda: random.uniform(-1, 1))
pset.renameArguments(ARG0="x")

Functions that tell how to create an invividual. genTree creates a tree with objects from the pset, individual sets the tree to the individualusing gentTree, and population rebeats the creation of the individual in a list, in order to create a population.

In [4]:
toolbox = base.Toolbox()
toolbox.register("genTree", gp.genFull, pset = pset, min_= 1, max_= 4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.genTree)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Function used for evaluation

In [5]:
def evaluate(individual, pset, points):
    function = gp.compile(expr=individual, pset=pset)
    MSE = np.array([ function(x) - misteriousFunction(x)  for x in points])**2
    return np.sum(MSE), 
    
def misteriousFunction(x):
    return x**3 -x**2 + x -5

evaluate registers the function used to calculate the fitness. I suspect that these names are standard and have to be used if using a pre-made alg. Mate does a one point crossover, the mutation adds a tree to a node with uniform probability, select selects the individuals for mating.

In [6]:
toolbox.register("evaluate", evaluate, points=[x/10. for x in range(-10, 10)], pset=pset)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, pset=pset, min_=0,max_= 2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)

Fixes the height of the tree, very important

In [7]:
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

Allows multiprocessing, much much faster.

In [11]:
import multiprocessing

pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

Stats module, logged in the loggbook, which will pretty print the stats

In [9]:
stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
stats.register("max", np.max)
stats.register("mean", np.mean)
stats.register("min", np.min)


In [15]:
import time
pop = toolbox.population(n=100)
start = time.time()
pop, logbook =algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=2000, verbose=False)
end = time.time()
print(end-start)

42.903300285339355
